In [ ]:
# Times Square parameters
import datetime
end_date = datetime.datetime.strptime('2024-01-08', "%Y-%m-%d")
lookback = 14

# OBS issue summary

In [ ]:
import os
import json
from urllib.parse import urljoin, urlencode
from typing import Any
from pprint import pprint
from datetime import datetime, timedelta
from collections import defaultdict

import httpx
import pandas as pd
from IPython.display import HTML
from lsst.rsp import get_access_token

In [ ]:
class JiraClient:
    """Jira API client for Notebook Aspect users."""

    def __init__(self) -> None:
        env_url = os.getenv("EXTERNAL_INSTANCE_URL", None)
        if env_url is None:
            raise RuntimeError("EXTERNAL_INSTANCE_URL environment variable is not set")

        self.jira_url = urljoin(env_url, "/jira-data-proxy/")
        self._http_client = httpx.AsyncClient()

    async def aclose(self) -> None:
        await self._http_client.close()

    @property
    def _token(self) -> str:
        """An RSP token."""
        return get_access_token()

    async def search(
        self,
        jql: str,
        start_at: int | None = None,
        max_results: int | None = None,
        fields: list[str] | None = None,
        exclude_fields: list[str] | None = None,
        expand: list[str] | None = None
    ) -> dict[str, Any]:
        # print(jql)
        qs_parts = [("jql", jql)]
        if start_at:
            qs_parts.append(("startAt", str(start_at)))
        if max_results:
            qs_parts.append(("maxResults", str(max_results)))
        if fields or exclude_fields:
            all_fields = []
            if fields:
                all_fields.extend(fields)
            if exclude_fields:
                all_fields.extend([f"-{f}" for f in exclude_fields])
            qs_parts.append(("fields", ",".join(all_fields)))
        if expand:
            qs_parts.append(("exclude", ",".join(expand)))
        qs = urlencode(qs_parts)
        url = urljoin(self.jira_url, f"rest/api/2/search?{qs}")
        # print(url)
        r = await self._http_client.get(url, headers={"Authorization": f"Bearer {self._token}"})
        r.raise_for_status()
        return r.json()


client = JiraClient()

In [ ]:
td = timedelta(days=lookback)
start_date = end_date - td
# print(end_date)
obs_data = await client.search(
    f'project = "OBS" AND created >= {start_date.strftime("%Y-%m-%d")} AND created <= {end_date.strftime("%Y-%m-%d")}',
    fields=["summary", "created", "status", "reporter", "components"]
)
# pprint(obs_data)

In [ ]:
reduced_obs_data = defaultdict(list)
for obs in obs_data["issues"]:
    reduced_obs_data["key"].append(obs["key"])
    fields = obs["fields"]
    reduced_obs_data["summary"].append(fields["summary"])
    reduced_obs_data["created"].append(datetime.strptime(fields["created"], "%Y-%m-%dT%H:%M:%S.%f%z"))
    reduced_obs_data["reporter"].append(fields["reporter"]["displayName"])
    reduced_obs_data["status"].append(fields["status"]["name"])

issues_df = pd.DataFrame(reduced_obs_data)
issues_df = issues_df.sort_values(by=["created"])

def link_issue(handle):
    return f'<a href="https://rubinobs.atlassian.net/browse/{handle}">{handle}</a>'


def format_timestamp(dt):
    return dt.strftime("%Y-%m-%d %H:%M")


HTML(
    issues_df
        .style
        .hide(axis="index")
        .format(format_timestamp, subset="created")
        .format(link_issue, subset="key")
        .set_table_styles(
            [{'selector': 'th', 'props': [('text-align', 'left')]}]
        )
        .set_properties(**{'text-align': 'left', 'min-width': '100px'})
        .set_properties(**{'min-width': '200px'}, subset="created")
        .set_caption(f"OBS issues {start_date.strftime("%Y-%m-%d")} – {end_date.strftime("%Y-%m-%d")}")
    .to_html()
)